In [1]:
import numpy as np
import torch

from ssd.utils_ssd300 import dboxes300_coco
from ssd.create_model import nvidia_ssd
from ssd.utils_ssd300 import Encoder
from ssd.model import Loss, SSD300
from ssd.SSD_Transformers import SSDTransformer
from utils.utils import set_seed
from train_model import train_model
from ssd.model_eval import model_evaluate
from torch.autograd import Variable

from torch.utils.data import DataLoader

from config.config import Configs
from ssd.model_eval import model_evaluate

from ssd.decode_results import Processing as processing
from ssd.loader_crowdhuman import CrowdHuman

from tqdm.auto import tqdm


c:\Users\123\Desktop\torch cuda\.venv\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
c:\Users\123\Desktop\torch cuda\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = 'cuda'
ssd_300 = nvidia_ssd(device=device, pretrainded_custom=False, pretrained_default=True)

feat_ext_w = ssd_300.feature_extractor.state_dict()

model = SSD300(label_num=3)
model = model.cuda()

model.feature_extractor.load_state_dict(feat_ext_w)


<All keys matched successfully>

In [5]:
batch_size = 32
device = "cuda"

for param in model.parameters():
    param.requires_grad = True

for param in model.feature_extractor.parameters():
    param.requires_grad = False

dboxes = dboxes300_coco()
encoder = Encoder(dboxes)

loss_func = Loss(dboxes, device=device)
transformers_ssd_train = SSDTransformer(dboxes, size=(300, 300), val=False)
transformers_ssd_val = SSDTransformer(dboxes, size=(300, 300), val=True)
set_seed()
data_train = CrowdHuman(
    img_folder="data\\train\\",
    annotate_file="data\\annotation_train.json",
    transform=transformers_ssd_train,
    use_head_bbx=True,
)


train_dataloader = DataLoader(
    data_train,
    batch_size=batch_size,
    shuffle=True 
)


data_val = CrowdHuman(
    img_folder="data\\val\\",
    annotate_file="data\\annotation_val.json",
    transform=transformers_ssd_val,
    use_head_bbx=True,
)

val_dataloader = DataLoader(
    data_val,
    batch_size=batch_size,
    shuffle=False,  
)


data_test = CrowdHuman(
    img_folder="data\\test\\",
    annotate_file="data\\annotation_test.json",
    transform=transformers_ssd_val,
    use_head_bbx=True,
)

test_dataloader = DataLoader(
    data_test,
    batch_size=batch_size,
    shuffle=False,  
)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.001)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, patience=4,threshold = 1e-3, verbose=True, factor=0.5, mode = 'max'
)

load annotation file:  data\annotation_train.json
load annotation file:  data\annotation_val.json
load annotation file:  data\annotation_test.json


In [7]:
ckpt = torch.load('weight\\state best_model_at_adam2.pth')

ckpt = ckpt["model"]
model.load_state_dict(ckpt)


<All keys matched successfully>

In [9]:
print("Evaluating")
loss_sum = 0
l1 =  Loss(dboxes, device=device)
for i, data in enumerate(tqdm(val_dataloader, total=len(val_dataloader))):
    model.eval()
    img, _, images_sizes, bbox_data, bbox_labels = data
    with torch.no_grad():
        ploc, plabel = model(img.to(device))
    new_bbox_data = []
    new_bbox_labels = []
    for i in zip(bbox_data, bbox_labels ):
        bbox_ = i[0][i[1] != 0]
        label_ = i[1][i[1] != 0]
        bbox, label = encoder.encode(bbox_, label_)
        new_bbox_data.append(bbox)
        new_bbox_labels.append(label)
    
    new_bbox_data = torch.stack(new_bbox_data, dim= 0)
    new_bbox_labels = torch.stack(new_bbox_labels, dim= 0)
    
    gloc = Variable(new_bbox_data.transpose(1, 2).contiguous(), requires_grad=False).to(
            device)
    glabel = Variable(new_bbox_labels, requires_grad=False).to(device)

    loss = l1(ploc=ploc, plabel=plabel, gloc=gloc, glabel=glabel)
    loss_sum += loss.item()       
    

Evaluating


100%|██████████| 47/47 [01:28<00:00,  1.89s/it]


In [10]:
loss_sum / len(val_dataloader)

3.7407786947615604